In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_ollama import ChatOllama

In [2]:
prompt = ChatPromptTemplate.from_messages(
    [SystemMessage(content='You are a helpful assistant that translates English to French.'), ('human', '{input}')]
)
model = ChatOllama(model='qwen2.5:3b', temperature=0.7)
chain = prompt | model
chain.invoke({'input': "Translate 'Hello, how are you?' to French."})

AIMessage(content="'Hello, how are you?' se traduit en français par 'Bonjour, et vous allez bien ?'", additional_kwargs={}, response_metadata={'model': 'qwen2.5:3b', 'created_at': '2025-05-30T07:43:58.4433746Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2284623000, 'load_duration': 1044088400, 'prompt_eval_count': 35, 'prompt_eval_duration': 363600700, 'eval_count': 24, 'eval_duration': 874043300, 'model_name': 'qwen2.5:3b'}, id='run--654f6526-0798-43cf-976b-3049af2aacb4-0', usage_metadata={'input_tokens': 35, 'output_tokens': 24, 'total_tokens': 59})

In [ ]:
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

config = {'configurable': {'session_id': 'abc2'}}


config = {'configurable': {'session_id': 'abc2'}}
response = with_message_history.invoke('who am i', config=config)


In [52]:
from langchain_core.prompts import MessagesPlaceholder

f = MessagesPlaceholder(variable_name='messages')

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """add two numbers"""
    return a + b


tools = [add]
llm_with_tool = model.bind_tools(tools=tools)

In [5]:
messages = []
question = HumanMessage(content='What is 1+1?')
messages.append(question)
response = llm_with_tool.invoke(messages)
print(response)

content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:3b', 'created_at': '2025-05-30T07:45:12.8437103Z', 'done': True, 'done_reason': 'stop', 'total_duration': 989427400, 'load_duration': 21213100, 'prompt_eval_count': 167, 'prompt_eval_duration': 39753700, 'eval_count': 25, 'eval_duration': 923720400, 'model_name': 'qwen2.5:3b'} id='run--7f09eef7-2a52-41e6-8883-5be48add860f-0' tool_calls=[{'name': 'add', 'args': {'a': 1, 'b': 1}, 'id': '6f5b6d73-7dec-4c4c-aa17-2dcf8fe06699', 'type': 'tool_call'}] usage_metadata={'input_tokens': 167, 'output_tokens': 25, 'total_tokens': 192}


In [ ]:
def call_tools(msg: AIMessage) -> list[dict]:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in [add]}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_msg = tool_map[tool_call['name']].invoke(tool_call)
        messages.append(tool_msg)
    return tool_msg['content']


In [6]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)


In [7]:
response = agent_executor.invoke({'messages': '1+1=?'})

response['messages']

[HumanMessage(content='1+1=?', additional_kwargs={}, response_metadata={}, id='5be60e37-826e-4064-a1f3-0ebf4dd37550'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:3b', 'created_at': '2025-05-30T07:45:20.3160289Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1065912900, 'load_duration': 22251200, 'prompt_eval_count': 164, 'prompt_eval_duration': 111129900, 'eval_count': 25, 'eval_duration': 927029300, 'model_name': 'qwen2.5:3b'}, id='run--f9f2eaf6-7898-4e57-a482-85cefe427411-0', tool_calls=[{'name': 'add', 'args': {'a': 1, 'b': 1}, 'id': 'd6ee72f9-0a98-44d2-89b0-693fbe1de14c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 164, 'output_tokens': 25, 'total_tokens': 189}),
 ToolMessage(content='2', name='add', id='af514a2f-1612-4863-a25a-fda79ff334a0', tool_call_id='d6ee72f9-0a98-44d2-89b0-693fbe1de14c'),
 AIMessage(content='The result of adding 1 and 1 is 2.', additional_kwargs={}, response_metadata={'model': 'qwen2.5:3b', 'created

In [ ]:
from sqlalchemy import create_engine

# 替换为你的数据库配置
MYSQL_HOST = '192.168.31.175'
MYSQL_PORT = 3306
MYSQL_USER = 'jhyl**hr'
MYSQL_PASSWORD = 'JHyl*#369'
MYSQL_DATABASE = 'wj2'

url = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}'
engine = create_engine(url)

# 测试连接
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(url)
print(db.get_usable_table_names())
